In [27]:
%use kotlin-statistics, krangl, lets-plot, numpy(0.1.4)

@file:Repository("https://repo1.maven.org/maven2")
@file:DependsOn("de.sldk:kotbar:0.1.0")

In [2]:
class Obj(
    val x: MutableList<Double>,
    val y: Int
)

class Batch(
    val ind: Int,
    val trainObjs: List<Obj>,
    val testObjs: List<Obj>
)

In [3]:
class ConfInfo(
        var tp: Int = 0,
        var fp: Int = 0,
        var fn: Int = 0,
        var tn: Int = 0,
        var cnt: Int = 0,
        var prec: Double = 0.0,
        var recall: Double = 0.0,
        var fSc: Double = 0.0
)

class Stat(
    val accuracy: Double, 
    val tpr: Double,
    val fpr: Double
)

fun getStat(confMatrix: Array<IntArray>): Stat {
    val k = confMatrix.size
    val infos = Array(k) { ConfInfo() }
    var all = 0
    for (i in 0 until k) {
        for (j in 0 until k) {
            val cur = confMatrix[i][j]
            all += cur
            infos[i].cnt += cur
            if (i == j) {
                infos[i].tp = cur
            } else {
                infos[i].fp += cur
                infos[j].fn += cur
            }
        }
    }
    infos.forEach {
        it.tn = all - it.fp - it.fn - it.tp
        it.recall = if (it.tp + it.fn != 0) it.tp.toDouble() / (it.tp + it.fn) else 0.0
        it.prec = if (it.tp + it.fp != 0) it.tp.toDouble() / (it.tp + it.fp) else 0.0
        it.fSc = if (it.recall + it.prec != 0.0) 2 * it.recall * it.prec / (it.recall + it.prec) else 0.0
    }
    val acc = infos.map{(it.tn + it.tp).toDouble() / (it.tn + it.tp + it.fn + it.fp)}.filter{it != 0.0}.average()
    val tpr = infos.map{it.tp}.sum().toDouble() / infos.map {it.tp + it.fn}.sum()
    val fpr = infos.map{it.fp}.sum().toDouble() / infos.map {it.fp + it.tn}.sum()
    return Stat(acc, tpr, fpr)
}

In [4]:
import java.io.File

fun normilize(objs: List<Obj>) {
    for(i in 0 until objs[0].x.size) {
        var min = Double.MAX_VALUE
        var max = Double.MIN_VALUE
        for (obj in objs) {
            if (obj.x[i] < min) {
                min = obj.x[i]
            }
            if (obj.x[i] > max) {
                max = obj.x[i]
            }
        }
        val b = max - min
        for (obj in objs) {
            obj.x[i] = if (b == 0.0) {
                0.0
            } else {
                (obj.x[i] - min) / b
            }
        }
    }
}

fun readData(): List<Batch> {
    val batches = ArrayList<Batch>()
    var train = ArrayList<Obj>()
    var test = ArrayList<Obj>()
    File("res").listFiles().sortedBy { it.name.take(2).toInt() }.forEach { f ->
        if (train.isNotEmpty() && test.isNotEmpty()) {
            batches.add(Batch(f.name.take(2).toInt() - 1, train, test))
            train = ArrayList()
            test =  ArrayList()
        }        
        val objs = if (f.name.contains("train")) train else test
        val df = DataFrame.readCSV(f)
        val X = df.remove("y").toDoubleMatrix()
        val Y = df.get("y").values().map{ it as Int}
        for(i in X[0].indices) {
            val x = ArrayList<Double>()
            for (j in X.indices) {
                x.add(X[j][i])
            }
            objs.add(Obj(x,Y[i] - 1))
        }
        normilize(objs)
    }
    
    batches.add(Batch(21, train, test))
    
    return batches
}


In [5]:
readData()[0].trainObjs.joinToString("|||\n") {it.x.joinToString(" ")}
// readData()[0].trainObjs.joinToString("\n") {it.y.toString()}

0.6275820576241727 0.3738142291222891 0.49698083144402627 0.4191823526122752 0.8885328564950721 0.319377990430622 0.4152621674704618 0.542956153293232 0.444464032361585|||
0.6693342755913784 0.4771745918973232 0.3717120162412158 0.592696753207898 0.4272335278185227 0.41847904886182397 0.2914982810095256 0.89317746755949 0.4893623534776881|||
0.2389268763190817 0.5337325124043464 0.4307322321372065 0.636016742574578 0.5313050176409205 0.5645208061476004 0.26334317082617803 0.461438353573185 0.5596993970205723|||
0.49420320304069215 0.4797202241182302 0.5497194890866922 0.42329736903800824 0.4958705302025002 0.8046251993620415 0.21715494222615264 0.43741686528203383 0.7568014643786103|||
0.2626130470906191 0.4287255039005241 0.7611766939545896 0.5730790622667218 0.5173000316196104 0.551761635493693 0.32953201641181734 0.3888790526992774 0.535525326824984|||
0.5326225015041113 0.4653479821536433 0.4624825184804635 0.41117401075299403 0.6047362712847119 0.45559663621864577 0.32151463112829

In [6]:
class Node(
    val id: Int,
    val y: Int,
    val leaf: Boolean,
    val splitValue: Double = 0.0,
    val splitInd: Int = 0,
    var left: Node? = null,
    var right: Node? = null
)

class GiniRes(
    val left: List<Obj>,
    val right: List<Obj>,
    val splitValue: Double,
    val splitInd: Int
)

var treeInd = 1
var K = 0
val treeList = ArrayList<Node>()

fun buildTree(objs: List<Obj>, depth: Int, maxD: Int, isForest: Boolean = false): Pair<Node, Int> {
    var d = depth
    terminalIfOneClass(objs)?.let {
        treeList.add(it)
        return Pair(it, d)
    }

    val giniRes = splitTree(objs, isForest)
    var node: Node
    if (giniRes.left.isEmpty() || giniRes.right.isEmpty()) {
        node = terminal(giniRes.left, giniRes.right)
    } else if (depth >= maxD) {
        node = terminal(giniRes.left, giniRes.right)
    } else {
        val ind = treeInd++
        val left = buildTree(giniRes.left, depth + 1, maxD)
        val right = buildTree(giniRes.right, depth + 1, maxD)
        d = max(d, right.second)
        d = max(d, left.second)
        node = Node(ind, 0, false, giniRes.splitValue, giniRes.splitInd, left.first, right.first)
    }
    treeList.add(node)
    return Pair(node, d)

}

fun getFeatures(isForest: Boolean, n: Int): List<Int> {
    if (!isForest) {
        return (0 until n).toList()
    }
    return (0 until n).toList().shuffled().take(sqrt(n.toDouble()).toInt())
}

 fun splitTree(o: List<Obj>, isForest: Boolean): GiniRes {
    var bestGini = 10000000000000000000.0
    var bestXInd = 0
    var splitValue = 0.0
    for (xInd in getFeatures(isForest, o[0].x.size)) {
        val objs = o.sortedByDescending { it.x[xInd] }
        val leftClassCount = MutableList(K) { 0 }
        val rightClassCount = MutableList(K) { 0 }
        val leftClassVals = MutableList(K) { 0.0 }
        val rightClassVals = MutableList(K) { 0.0 }
        for (obj in objs) {
            leftClassCount[obj.y]++
        }
        for (cl in 0 until K) {
            leftClassVals[cl] = (1.0 * leftClassCount[cl] / objs.size).pow(2)
        }
        for (i in -1 until objs.size) {
            if (i != -1) {
                val obj = objs[i]
                leftClassCount[obj.y]--
                for (j in leftClassVals.indices) {
                    leftClassVals[j] = (1.0 * leftClassCount[j] / (objs.size - (i + 1))).pow(2)
                }
                rightClassCount[obj.y]++
                for (j in rightClassVals.indices) {
                    rightClassVals[j] = (1.0 * rightClassCount[j] / (i + 1)).pow(2)
                }
            }
            val giniLeft = 1.0 - leftClassVals.sum()
            val giniRight = 1.0 - rightClassVals.sum()
            val resGini =
                (objs.size - (i + 1)).toDouble() / objs.size * giniLeft + (i + 1).toDouble() / objs.size * giniRight
            if (resGini < bestGini) {
                bestGini = resGini
                bestXInd = xInd
                splitValue = if (i == -1) {
                    Int.MAX_VALUE.toDouble()
                } else if (i == objs.size - 1) {
                    Int.MIN_VALUE.toDouble()
                } else {
                    (objs[i].x[xInd].toDouble() + objs[i + 1].x[xInd].toDouble()) / 2.0
                }
               // splitValue = if (i < 0) Int.MAX_VALUE.toDouble() else objs[i].x[xInd].toDouble()
            }
        }
    }

    val left = ArrayList<Obj>()
    val right = ArrayList<Obj>()

    for (obj in o) {
        if (obj.x[bestXInd] < splitValue) {
            left.add(obj)
        } else {
            right.add(obj)
        }
    }
    return GiniRes(left, right, splitValue, bestXInd)
}

fun terminalIfOneClass(objs: List<Obj>): Node? {
    if (objs.map { it.y }.toHashSet().size == 1) {
        return Node(treeInd++, objs[0].y, true)
    }
    return null
}

fun terminal(objs: List<Obj>): Node {
    return terminal(objs, emptyList())
}

fun terminal(left: List<Obj>, right: List<Obj>): Node {
    val classCount = MutableList(K) { 0 }
    for (obj in left) {
        classCount[obj.y]++
    }
    for (obj in right) {
        classCount[obj.y]++
    }
    var maxCount = 0
    var maxCountClass = 0
    for (i in classCount.indices) {
        if (classCount[i] > maxCount) {
            maxCount = classCount[i]
            maxCountClass = i
        }
    }
    return Node(treeInd++, maxCountClass, true)
}

fun predict(obj: Obj, tree: Node): Int {
    if (tree.leaf) {
        return tree.y
    } else {
        if (obj.x[tree.splitInd] < tree.splitValue) {
            return predict(obj, tree.left!!)
        } else {
            return predict(obj, tree.right!!)
        }
    }
}

In [24]:
val depthParam = (0..10).toList()

class Res(
    val bestHeight: Int,
    val acc: Double,
    val plotData: Map<Int, Double>
)

fun calc(batch: Batch, plot: Boolean = false): Res {
    var bestDepth = 0
    var bestAcc = 0.0
    
    var plotData = HashMap<Int,Double>()
    val maxClass = max(batch.trainObjs.map{it.y}.max()!!, batch.testObjs.map{it.y}.max()!!)
    K = maxClass + 1
    for(d in depthParam) {
        val confMatrix = Array(maxClass + 1) {IntArray(maxClass + 1)}
        val (tree, curD) = buildTree(batch.trainObjs, 0, d)
        val testRes = batch.testObjs.map {
            predict(it, tree)
        }
        for (j in batch.testObjs.indices) {
            confMatrix[testRes[j]][batch.testObjs[j].y] += 1
           // println(testRes[j] == batch.testObjs[j].y )
        }
        val acc = getStat(confMatrix).accuracy
        if (acc > bestAcc) {
            bestAcc = acc
            bestDepth = curD
        }
        plotData[d] = acc
    }
    if (plot) {
        plot(plotData)
    }
    return Res(bestDepth, bestAcc, plotData)
}

fun plot(p: HashMap<Int,Double>) {
    val dat = p.toList().sortedBy { it.first }
    val plotData = mapOf<String, Any>(
    "depth" to dat.map {it.first},
    "accuracy" to dat.map{it.second}
    )
    var p1 = lets_plot(plotData){x = "depth"; y = "accuracy"}
    p1+= geom_path() + geom_point()
    p1.show()
}

In [8]:
val results = readData().map { batch ->
    calc(batch)
}

In [9]:
results.forEachIndexed {i, r ->
    print("""
        dataset num: ${i}
        best accuracy: ${r.acc}
        depth: ${r.bestHeight}
    """)
}


        dataset num: 0
        best accuracy: 0.9815728604471857
        depth: 3
    
        dataset num: 1
        best accuracy: 0.9168134664317871
        depth: 9
    
        dataset num: 2
        best accuracy: 0.9969262295081966
        depth: 1
    
        dataset num: 3
        best accuracy: 0.9655335968379447
        depth: 5
    
        dataset num: 4
        best accuracy: 0.998917748917749
        depth: 1
    
        dataset num: 5
        best accuracy: 0.9010301692420898
        depth: 4
    
        dataset num: 6
        best accuracy: 0.9609302325581394
        depth: 3
    
        dataset num: 7
        best accuracy: 0.9792099792099792
        depth: 2
    
        dataset num: 8
        best accuracy: 0.8923747276688453
        depth: 4
    
        dataset num: 9
        best accuracy: 0.9597048960429241
        depth: 4
    
        dataset num: 10
        best accuracy: 0.9884909456740443
        depth: 1
    
        dataset num: 11
        best accur

График на тестовом множестве, датасет №1, глубина = 9

In [30]:
calc(readData()[1], true)

Line_51_jupyter$Res@27e9c4e5

График на обучающем множестве, датасет №1, глубина = 9

In [28]:
val batch = readData()[1].let {
    Batch(it.ind, it.trainObjs, it.trainObjs)
}
calc(batch, true) 

Line_51_jupyter$Res@6900bd9f

График на тестовом множестве, датасет №14, глубина = 0

In [29]:
val a2 = calc(readData()[14], true)

График на обучающем множестве, датасет №14, глубина = 0

In [31]:
val batch = readData()[14].let {
    Batch(it.ind, it.trainObjs, it.trainObjs)
}
calc(batch, true) 

Line_51_jupyter$Res@3a1b564b

In [19]:
fun buildForest(objs: List<Obj>): List<Node> {
    val res = ArrayList<Node>()
    repeat(100) {
        val pickedObjs = ArrayList<Obj>()
        repeat(objs.size) {
            pickedObjs.add(objs.random())
        }
        res.add(buildTree(pickedObjs, 0, 1000000000, true).first)
    }
    return res
}

fun predictForest(obj: Obj, forest: List<Node>): Int {
    val res = HashMap<Int, Int>()
    forest.forEach { t ->
        val cl = predict(obj, t)
        res.putIfAbsent(cl, 0)
        res[cl] = res[cl]!! + 1
    }
    return res.toList().sortedByDescending { it.second }[0].first
}

fun calcForest(batch: Batch): Pair<Double, Double> {
    val maxClass = max(batch.trainObjs.map{it.y}.max()!!, batch.testObjs.map{it.y}.max()!!)
    K = maxClass + 1
    val forest = buildForest(batch.trainObjs)
    
    val testRes = batch.testObjs.map {
        predictForest(it, forest)
    }
    var confMatrix = Array(maxClass + 1) {IntArray(maxClass + 1)}
    for (j in batch.testObjs.indices) {
        confMatrix[testRes[j]][batch.testObjs[j].y] += 1
    }
    val accTest = getStat(confMatrix).accuracy
    
    val trainRes = batch.trainObjs.map {
        predictForest(it, forest)
    }
    confMatrix = Array(maxClass + 1) {IntArray(maxClass + 1)}
    for (j in batch.trainObjs.indices) {
        confMatrix[trainRes[j]][batch.trainObjs[j].y] += 1
    }
    val accTrain = getStat(confMatrix).accuracy
    
    return Pair(accTrain, accTest)
}

In [20]:
val forestResults = readData().map {
    calcForest(it)
}

In [21]:
forestResults.forEachIndexed { i, r ->
    print("""
    dataset num: ${i}
    accuracy on train set: ${r.first}
    accuracy on test set: ${r.second}
""")
}


    dataset num: 0
    accuracy on train set: 1.0
    accuracy on test set: 0.9811102544333075

    dataset num: 1
    accuracy on train set: 1.0
    accuracy on test set: 0.9229594832648268

    dataset num: 2
    accuracy on train set: 1.0
    accuracy on test set: 0.9956002892960464

    dataset num: 3
    accuracy on train set: 0.9999367588932806
    accuracy on test set: 0.9656600790513834

    dataset num: 4
    accuracy on train set: 1.0
    accuracy on test set: 0.9924242424242424

    dataset num: 5
    accuracy on train set: 1.0
    accuracy on test set: 0.8995584988962473

    dataset num: 6
    accuracy on train set: 1.0
    accuracy on test set: 0.9598308668076108

    dataset num: 7
    accuracy on train set: 1.0
    accuracy on test set: 0.9604989604989606

    dataset num: 8
    accuracy on train set: 1.0
    accuracy on test set: 0.8971677559912854

    dataset num: 9
    accuracy on train set: 1.0
    accuracy on test set: 0.9591146881287725

    dataset num: 10
    